In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
print(torch.__version__)
!pip install dgl-cu111 dglgo -f https://data.dgl.ai/wheels/repo.html
!pip install neuralkg
!git clone https://github.com/zjukg/NeuralKG.git

1.10.0+cu111
Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 252.7 MB 58 kB/s 
     |████████████████████████████████| 77 kB 3.3 MB/s 
     |████████████████████████████████| 103 kB 29.7 MB/s 
     |████████████████████████████████| 596 kB 46.7 MB/s 
     |████████████████████████████████| 10.9 MB 34.5 MB/s 
     |████████████████████████████████| 109 kB 48.3 MB/s 
     |████████████████████████████████| 51 kB 7.8 MB/s 
     |████████████████████████████████| 45 kB 3.4 MB/s 
     |████████████████████████████████| 42 kB 958 kB/s 
     |████████████████████████████████| 546 kB 47.6 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import neuralkg
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
import wandb
from neuralkg.utils import setup_parser
from neuralkg.utils.tools import *
from neuralkg.data.Sampler import *
import yaml

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
parser = setup_parser()
args = parser.parse_args(args=[])
#specify the config path
config_path = "drive/MyDrive/Colab Notebooks/neuralkg_guide.yaml"

#load config to args
args = load_config(args, config_path)
args.data_path = "NeuralKG/dataset/FB15K237"
#set random seeds
seed_everything(args.seed)
#set up sampler,datamodule,model,litmodel dynamicly
train_sampler = import_class(f"neuralkg.data.{args.train_sampler_class}")(args)
test_sampler = import_class(f"neuralkg.data.{args.test_sampler_class}")(train_sampler)
kgdata = import_class(f"neuralkg.data.{args.data_class}")(args, train_sampler, test_sampler)
model = import_class(f"neuralkg.model.{args.model_name}")(args)
lit_model = import_class(f"neuralkg.lit_model.{args.litmodel_name}")(model, args)
#set up logger, TensorBoardLogger is used by default
logger = pl.loggers.TensorBoardLogger("training/logs")
if args.use_wandb:
    log_name = "_".join([args.model_name, args.dataset_name, str(args.lr)])
    logger = pl.loggers.WandbLogger(name=log_name, project="NeuralKG")
    logger.log_hyperparams(vars(args))
#set up early_callback to early stopping
early_callback = pl.callbacks.EarlyStopping(
        monitor="Eval|mrr",
        mode="max",
        patience=args.early_stop_patience,
        check_on_train_epoch_end=False,
    )
#set up model saving method
dirpath = "/".join(["output", args.eval_task, args.dataset_name, args.model_name])
model_checkpoint = pl.callbacks.ModelCheckpoint(
    monitor="Eval|mrr",
    mode="max",
    filename="{epoch}-{Eval|mrr:.3f}",
    dirpath=dirpath,
    save_weights_only=True,
    save_top_k=1,
)
callbacks = [early_callback, model_checkpoint]
# initialize trainer
trainer = pl.Trainer.from_argparse_args(
        args,
        callbacks=callbacks,
        logger=logger,
        default_root_dir="training/logs",
        gpus="0,",
        check_val_every_n_epoch=args.check_per_epoch,
    )


Global seed set to 666
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
!nvidia-smi


✌ We show here the training process of TransE on FB15K-237. Note that only the complete training and testing process is shown here due to the limitation of memory and efficiency, to get reproduced results, please use the configuration of the best parameters for reproduction under the [config file](https://github.com/zjukg/NeuralKG/tree/main/config)

In [ ]:
#train&eval model
trainer.fit(lit_model, datamodule=kgdata)
#test model
lit_model.eval()
trainer.test(lit_model, datamodule=kgdata)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: training/logs/default

  | Name  | Type     | Params
-----------------------------------
0 | model | TransE   | 3.0 M 
1 | loss  | Adv_Loss | 3.0 M 
-----------------------------------
3.0 M     Trainable params
2         Non-trainable params
3.0 M     Total params
11.822    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Global seed set to 666


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/data.py:60: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 10. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"


...

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'Test|hits@1': 0.03999999910593033,
 'Test|hits@10': 0.20999999344348907,
 'Test|hits@3': 0.11999999731779099,
 'Test|mrr': 0.10000000149011612}
--------------------------------------------------------------------------------


[{'Test|hits@1': 0.03999999910593033,
  'Test|hits@10': 0.20999999344348907,
  'Test|hits@3': 0.11999999731779099,
  'Test|mrr': 0.10000000149011612}]